In [79]:
import pandas as pd
from  sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,  plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,  f1_score, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
import numpy as np


In [81]:
df = pd.read_csv('Image_Segmentation.csv')
df

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,BRICKFACE,188,133,9,0.000000,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,BRICKFACE,105,139,9,0.000000,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,BRICKFACE,34,137,9,0.000000,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,BRICKFACE,39,111,9,0.000000,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,BRICKFACE,16,128,9,0.000000,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,GRASS,36,243,9,0.111111,0.0,1.888889,1.851851,2.000000,0.711110,13.333333,9.888889,12.111111,18.000000,-10.333333,-3.666667,14.000000,18.000000,0.452229,2.368311
205,GRASS,186,218,9,0.000000,0.0,1.166667,0.744444,1.166667,0.655555,13.703704,10.666667,12.666667,17.777779,-9.111111,-3.111111,12.222222,17.777779,0.401347,2.382684
206,GRASS,197,236,9,0.000000,0.0,2.444444,6.829628,3.333333,7.599998,16.074074,13.111111,16.666668,18.444445,-8.888889,1.777778,7.111111,18.555555,0.292729,2.789800
207,GRASS,208,240,9,0.111111,0.0,1.055556,0.862963,2.444444,5.007407,14.148149,10.888889,13.000000,18.555555,-9.777778,-3.444444,13.222222,18.555555,0.421621,2.392487


In [83]:
df.Class.value_counts()

Class
SKY          30
FOLIAGE      30
CEMENT       30
WINDOW       30
PATH         30
GRASS        30
BRICKFACE    29
Name: count, dtype: int64

In [85]:
X,y = df.drop('Class',axis=1) , df['Class']

In [87]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.3, random_state=24, stratify=y)

In [90]:
# #Tuning Hyper parameter for Logistic Regression 

# penalties = ['l1', 'l2', 'elasticnet', None]
# l1 = np.linspace(0.001,1,5)
# multi = ['ovr', 'multinomial']
# scores = []

# scaler = MinMaxScaler()


# for i in penalties:
#     for l in l1:
#         for m in multi:
#             lr = LogisticRegression(random_state= 24 , penalty=i , solver='saga', l1_ratio=l, multi_class=m)
#             lr.fit(X_train, Y_train)
#             y_pred = lr.predict(X_test)
#             scores.append([i, l ,m ,  accuracy_score(Y_test,y_pred)])

# scores        

# df1 = pd.DataFrame(scores , columns=['Penalty', 'l1', 'Multiclass','Accuracy'])
# df1.sort_values('Accuracy',ascending=False ).iloc[0]  

In [94]:
#Using SVM-linear , SVM-radial , LDA , Logistic Regression using Voting Classifier

svm_l = SVC(kernel='linear')
svm_r = SVC(kernel='rbf')
LDA = LinearDiscriminantAnalysis()
LR = LogisticRegression()



voting = VotingClassifier([('SVM-L' , svm_l) , ('SVM-R', svm_r), ('LDA', LDA), ('LR', LR)])


voting.fit(X_train,Y_train)
y_Pred = voting.predict(X_test)
print(accuracy_score(Y_test,y_Pred))
print(f1_score(Y_test,y_Pred,average='macro'))

0.8888888888888888
0.8854427736006684


C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [71]:
#To check whether voting imapacts outcome
# 1. SVM linear 

svm_l.fit(X_train,Y_train)
y_Pred = svm_l.predict(X_test)
print(accuracy_score(Y_test,y_Pred))
print(f1_score(Y_test,y_Pred,average='macro'))

0.8571428571428571
0.8517126148705095


In [73]:
#2 SVM for RBF 
svm_r.fit(X_train,Y_train)
y_Pred = svm_r.predict(X_test)
print(accuracy_score(Y_test,y_Pred))
print(f1_score(Y_test,y_Pred,average='macro'))

0.6825396825396826
0.6539682539682541


In [75]:
#For LDA 
LDA.fit(X_train,Y_train)
y_Pred = LDA.predict(X_test)
print(accuracy_score(Y_test,y_Pred))
print(f1_score(Y_test,y_Pred,average='macro'))

0.873015873015873
0.8719347388078038


In [77]:
# For LR 
LR.fit(X_train,Y_train)
y_Pred = LR.predict(X_test)
print(accuracy_score(Y_test,y_Pred))
print(f1_score(Y_test,y_Pred,average='macro'))

0.8571428571428571
0.8542229012817248


C:\Users\dbda.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
